# Logit Regression Laboratory

Un estudio sobre condiciones dermatológicas con datos del ICM Machine Learning Repository.

In [590]:
# Above sentences help auto reload modules 

%load_ext autoreload
%autoreload 2

# Import system libraries
from random import shuffle
from typing import List
import importlib
import numpy
import pandas as pd
import plotly.express as px

# Import custom libraries

import sys
sys.path.append('./src/cleaning')
sys.path.append('./src/learning')

import OutlinerUtils
import MissingValues
import Preprocess
import Distances
import Categorization

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [671]:
# Here we store all global variables

class Store:
    tag_column: str = "Class"
    
    data_raw: pd.DataFrame = None
    data_tmp: pd.DataFrame = None

    data_train: pd.DataFrame = None
    data_test: pd.DataFrame = None
    
    missing_indexes: List[int] = []
    outliner_indexes: List[int] = []
    tags: List[str] = []

store = Store()

In [672]:
# Import data

store.data_raw = pd.read_csv('./data/dermatology/dermatology-with-labels.data')
store.data_raw

,C0,C1,C2,C3,C4,C5,C6,C7,C8,C9,...,C25,C26,C27,C28,C29,C30,C31,C32,C33,Class
0,2,2,0,3,0,0,0,0,1,0,...,0,0,3,0,0,0,1,0,55,2
1,3,3,3,2,1,0,0,0,1,1,...,0,0,0,0,0,0,1,0,8,1
2,2,1,2,3,1,3,0,3,0,0,...,0,2,3,2,0,0,2,3,26,3
3,2,2,2,0,0,0,0,0,3,2,...,3,0,0,0,0,0,3,0,40,1
4,2,3,2,2,2,2,0,2,0,0,...,2,3,2,3,0,0,2,3,45,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,2,1,1,0,1,0,0,0,0,0,...,0,0,1,0,0,0,2,0,25,4
362,3,2,1,0,1,0,0,0,0,0,...,1,0,1,0,0,0,2,0,36,4
363,3,2,2,2,3,2,0,2,0,0,...,0,3,0,3,0,0,2,3,28,3
364,2,1,3,1,2,3,0,2,0,0,...,0,2,0,1,0,0,2,3,50,3


## Data Cleaning

2. Encontrar valores atípicos o faltantes entre las columnas de los
registros. En caso de que existan, cuantificarlos (contarlos) para
conocer el número total de ellos.

4. Encontrar los índices de los atípicos (si existen) y cuantificar
cuántos registros contienen estos tipos de datos.

In [673]:
# Get indexes for rows with missing values

store.missing_indexes = MissingValues.get_missing_values_indexes(store.data_raw)
print (f'Filas con valores faltantes: {len(store.missing_indexes)}')
print (store.missing_indexes)

Filas con valores faltantes: 8
[33, 34, 35, 36, 262, 263, 264, 265]


In [674]:
# Remove rows with missing values

store.data_raw = store.data_raw.drop(labels=store.missing_indexes, axis=0)

# Make sure all rows are numeric types

store.data_raw = store.data_raw.astype({'C33': int})

# Get outliners

store.outliner_indexes = OutlinerUtils.get_outliner_indexes(store.data_raw)
print (f'Valores atípicos: {len(store.outliner_indexes)}')

Valores atípicos: 315


## Data Preprocessing

5. En caso que la cantidad de registros que se eliminarían supera
el 40 % del total del conjunto de datos, se procederá a eliminar
únicamente el 20 % de forma aleatoria los registros atípicos.

In [677]:
print(len(store.data_raw.index))
store.data_tmp = OutlinerUtils.delete_random_amount(
    store.data_raw
    )
print(len(store.data_tmp.index))

358
287


In [678]:
# Group by tag

store.tags = [1, 2, 3, 4, 5, 6]
data_groups = Preprocess.separate_data_by_tags(
    store.data_tmp,
    store.tags,
    store.tag_column
)
count = 0
for tag in store.tags:
    print (f"Clase {tag} contiene {len(data_groups[tag].index)} registros")
    count += len(data_groups[tag].index)

print (f"{count} registros en total")

Clase 1 contiene 91 registros
Clase 2 contiene 51 registros
Clase 3 contiene 57 registros
Clase 4 contiene 41 registros
Clase 5 contiene 31 registros
Clase 6 contiene 16 registros
287 registros en total


3. Eliminar las clases que tienen menos de 50 registros( 50 casos
por enfermedad)

In [679]:
Preprocess.drop_rows_less_50(
    store.data_tmp,
    store.tags,
    data_groups
)
count = 0
for tag in store.tags:
    print (f"Clase {tag} contiene {len(data_groups[tag].index)} registros")
    count += len(data_groups[tag].index)
    
print (f"{count} registros en total")

Clase 1 contiene 91 registros
Clase 2 contiene 51 registros
Clase 3 contiene 57 registros
199 registros en total


In [680]:
Preprocess.balance_data_by_dropping_rows(
    store.data_tmp,
    store.tags,
    data_groups
)

count = 0
for tag in store.tags:
    print (f"Clase {tag} contiene {len(data_groups[tag].index)} registros")
    count += len(data_groups[tag].index)

print (f"{count} registros en total")

Clase 1 contiene 51 registros
Clase 2 contiene 51 registros
Clase 3 contiene 51 registros
153 registros en total


In [681]:
store.data_train, store.data_test = Preprocess.get_training_and_testing_groups(
    0.7,
    store.tags,
    data_groups
)

print (f"data_train {len(store.data_train.index)} filas. Aprox ~{int(len(store.data_train.index) / len(store.tags))} filas por clase.")
print (f"data_test {len(store.data_test.index)} filas. Aprox ~{int(len(store.data_test.index) / len(store.tags))} filas por clase.")
print (f"{len(store.data_train.index) + len(store.data_test.index)} registros en total")

data_train 105 filas. Aprox ~35 filas por clase.
data_test 48 filas. Aprox ~16 filas por clase.
153 registros en total


D:\Universidad\Analítica de datos\parcial1\lab-env\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


## Classification Model

In [682]:
point = pd.DataFrame([store.data_train.iloc[15]])

pointDistances = Distances.sort_distances(
    Distances.point_distances_against_others(
        point,
        store.data_train,
        store.tag_column,
        []
    )
)

for i, dist in enumerate(pointDistances):
    if i > 10:
        break
    print (f"Distancia a punto {dist.index}: {dist.distance}")

Distancia a punto 15: 0.0
Distancia a punto 17: 4.69041575982343
Distancia a punto 11: 4.795831523312719
Distancia a punto 22: 6.082762530298219
Distancia a punto 68: 6.082762530298219
Distancia a punto 6: 6.48074069840786
Distancia a punto 26: 6.48074069840786
Distancia a punto 0: 6.708203932499369
Distancia a punto 25: 6.708203932499369
Distancia a punto 29: 7.211102550927978
Distancia a punto 76: 7.280109889280518


In [685]:
dist_groups = Distances.group_point_distances_by_tags(pointDistances, store.tags)
Distances.print_contestants_side_by_side (dist_groups, store.tags, 10)

t: 1.000  2.000  3.000 
0  0.000  6.083  7.280 
1  4.690  7.810  7.616 
2  4.796  8.124  8.367 
3  6.083  9.899  10.863 
4  6.481  10.247  11.091 
5  6.481  10.863  11.314 
6  6.708  10.909  11.790 
7  6.708  11.832  12.490 
8  7.211  14.387  13.077 
9  7.483  16.125  13.454 


In [686]:
point = pd.DataFrame([store.data_test.iloc[0]])
print (point)

Categorization.categorize_point(
    store.data_train,
    point,
    10, # k
    store.tags,
    store.tag_column
)

     C0  C1  C2  C3  C4  C5  C6  C7  C8  C9  ...  C25  C26  C27  C28  C29  \
309   2   3   2   0   1   0   0   0   0   1  ...    2    0    0    0    0   

     C30  C31  C32  C33  Class  
309    0    2    0   43      1  

[1 rows x 35 columns]


1

## TO DO
6. Se debe imprimir la matriz de confusión por cada validación del
numeral anterior

## TO DO
7. El desempeño de este modelo debe ser encontrado al calcular
las métricas de sensibilidad, especificidad y precisión. 

## TO DO
8. Gráfico de curva ROC

## TO DO
9. Elegir modelo con el mejor desempeño posible

In [553]:
import matplotlib.pyplot as plt
import numpy
from sklearn import metrics

# Generar datos de ejemplo de etiquetas reales y predichas
actual = numpy.random.binomial(1,.9,size = 1000)
predicted = numpy.random.binomial(1,.9,size = 1000)

# Crear la matriz de confusión utilizando sklearn
confusion_matrix = metrics.confusion_matrix(actual, predicted)

# Crear la visualización de la matriz de confusión
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

# Graficar la matriz de confusión
cm_display.plot()
plt.show()

ModuleNotFoundError: No module named 'matplotlib'